In [16]:
import json
import requests
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import datetime
load_dotenv()


True

In [1]:
import urllib.request, json
from dotenv import load_dotenv
load_dotenv()
import os

#Google MapsDdirections API endpoint
endpoint = 'https://maps.googleapis.com/maps/api/directions/json?'
api_key = os.getenv("GOOGS")
#Asks the user to input Where they are and where they want to go.
origin = input('Where are you?: ').replace(' ','+')
destination = input('Where do you want to go?: ').replace(' ','+')
#Building the URL for the request
nav_request = 'origin={}&destination={}&key={}'.format(origin,destination,api_key)
request = endpoint + nav_request
#Sends the request and reads the response.
response = urllib.request.urlopen(request).read()
#Loads response as JSON
directions = json.loads(response)
directions

Where are you?: Madrid, Spain
Where do you want to go?: Lisbon, Portugal


{'geocoded_waypoints': [{'geocoder_status': 'OK',
   'place_id': 'ChIJgTwKgJcpQg0RaSKMYcHeNsQ',
   'types': ['locality', 'political']},
  {'geocoder_status': 'OK',
   'place_id': 'ChIJO_PkYRozGQ0R0DaQ5L3rAAQ',
   'types': ['locality', 'political']}],
 'routes': [{'bounds': {'northeast': {'lat': 40.4222771, 'lng': -3.6984884},
    'southwest': {'lat': 38.5785932, 'lng': -9.1782007}},
   'copyrights': 'Map data ©2020 Inst. Geogr. Nacional',
   'legs': [{'distance': {'text': '626 km', 'value': 625667},
     'duration': {'text': '6 hours 3 mins', 'value': 21775},
     'end_address': 'Lisbon, Portugal',
     'end_location': {'lat': 38.7223249, 'lng': -9.1393372},
     'start_address': 'Madrid, Spain',
     'start_location': {'lat': 40.4166876, 'lng': -3.7037766},
     'steps': [{'distance': {'text': '0.1 km', 'value': 126},
       'duration': {'text': '1 min', 'value': 46},
       'end_location': {'lat': 40.4167229, 'lng': -3.702288},
       'html_instructions': 'Head <b>east</b> on <b>Puer

In [14]:
html_string = ''

for num in directions["routes"][0]["legs"][0]["steps"]:
    html_string += num['html_instructions']+"\n"
    
print(html_string)

Head <b>east</b> on <b>Puerta del Sol</b> toward <b>Calle de Carretas</b><div style="font-size:0.9em">May be closed at certain times or days</div>
Continue onto <b>Carrera de S. Jerónimo</b>
Continue straight onto <b>Calle de Cedaceros</b>
Turn <b>left</b> onto <b>Calle de Alcalá</b>
Turn <b>right</b> onto <b>Calle de la Virgen de los Peligros</b>
Continue onto <b>Calle del Clavel</b>
Turn <b>left</b> onto <b>C/<wbr/> Gran Vía</b>
Turn <b>left</b> onto <b>Plaza de España</b>
Continue onto <b>Cuesta de San Vicente</b>
Keep <b>left</b> to continue on <b>Cuesta de San Vicente</b>/<wbr/><b>Paseo Embarcadero</b>/<wbr/><b>A-5</b><div style="font-size:0.9em">Continue to follow A-5</div>
Keep <b>left</b> to stay on <b>A-5</b>
Continue onto <b>A6</b><div style="font-size:0.9em">Entering Portugal</div>
Continue straight to stay on <b>A6</b><div style="font-size:0.9em">Toll road</div>
Continue onto <b>IP7</b><div style="font-size:0.9em">Toll road</div>
Merge onto <b>A2</b>/<wbr/><b>IP7</b><div st

In [29]:
def geocode(location):
    res = requests.get(f"https://geocode.xyz/{location}", params={"json":1})
    data = res.json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

def getCoords(location):
  #  try:
    my_coordinates = geocode(location)
    return my_coordinates["coordinates"]
#    except:
 #       return -4.29151, 43.3856

In [7]:
email = os.getenv("EMAIL")
token = os.getenv("TOKEN_SURFCHECK")

def getMoreBeaches(lat, longt, experience_input):

    """
        Firstly, this function is in charge of the API request to get the closest beaches to the
        location inputted by the user. Secondly, this function ranks the beaches from best to worst, 
        based on some parameters such as the experience level inputted by the user, the frequency of the waves,
        and the quality of the waves.
    """
    # URL for the API request
    url = f"https://surfvideos.xyz/locations.json?latitude={lat}&longitude={longt}&distance=10"

    # Credentials needed for the API request
    payload = {}
    headers = {
    'X-User-Email': email,
    'X-User-Token': token,
    'Accept': '{{Accept}}',
    'Content-Type': '{{Content}}'
    }

    response = requests.request("GET", url, headers=headers, data = payload)
    data = response.json()
    
    # IMPORTANT STEP: Creating a dataframe with one row per beach. Each column is one param, check below
    df = pd.DataFrame(
    [
        data[i]["data"]["attributes"]["name"], 
        data[i]["data"]["attributes"]["address"], 
        data[i]["data"]["attributes"]["direction"], 
        data[i]["data"]["attributes"]["experience"], 
        data[i]["data"]["attributes"]["wave_quality"], 
        data[i]["data"]["attributes"]["frequency"], 
        data[i]["data"]["attributes"]["bottom"], 
        data[i]["data"]["attributes"]["best_wind_direction"], 
        data[i]["data"]["attributes"]["best_swell_direction"],
        data[i]["data"]["attributes"]["latitude"],
        data[i]["data"]["attributes"]["longitude"]
] for i in range(len(data)))
    
    # Replacing the names of the columns with real, recognizeable names
    df = df.rename(columns={0: 'Name', 1: 'Address', 2: 'Direction', 3: 'Experience', 4: "Wave quality", 5: "Frequency", 6: 'Bottom', 7: 'Best wind', 8: 'Best swell', 9: "Latitude", 10:"Longitude"})
    df["latlong"] = None
  
    # Creating another column with lat and long joined together. This will be needed for a second API we use
    for i in range(df.shape[0]):
        df["latlong"][i] = f"{df['Latitude'][i]},{df['Longitude'][i]}"  

    # Creating columns with ints instead of strings: The higher the int, the better the beach
    # This will help us in getting a score for each beach
    cleanup_cols = {"Wave quality_num":     {"Totally Epic": 4, "World Class": 3, "Regional Classic": 2, "Normal": 1, "Sloppy": 0},
                "Frequency_num": {"Very consistent (150 day/year)": 4, "Regular": 3, "Sometimes break": 2, "Rarely break (5day/year)": 1,
                                  "Don't know": 1 }}

    #df = df[df['Wave quality'].notna()]
    #df = df[df['Frequency'].notna()]

    df["Wave quality_num"] = df["Wave quality"]
    df["Frequency_num"] = df["Frequency"]

    df.replace(cleanup_cols, inplace=True)

    # Getting a score in column 'Score'
    df["Score"] = df["Wave quality_num"] + df["Frequency_num"]
    df = df.drop(['Wave quality_num', 'Frequency_num'], axis=1)

    # Sorting rows by Score. If they have the same score, they are sorted by how far away they are from the user
    df = df.sort_values(by=['Score'], ascending=False)

    # This series of conditions sort the dataframe based the experience of the user
    if experience_input == "Beginner":
        new_df = df[(df['Experience'] == "Beginners wave")|(df['Experience'] == "All surfers")]
    elif experience_input == "Experienced":
        new_df = df[(df['Experience'] == "All surfers")|(df['Experience'] == "Experienced surfers")]
    elif experience_input == "Advanced/Pro":
        new_df = df[(df['Experience'] == "All surfers")|(df['Experience'] == "Experienced surfers")|(df['Experience'] == "Pros or kamikaze only...")]

    return new_df.head()
 


In [29]:
key = os.getenv("KEY")

def getSwellHeight(latlong):
    key = os.getenv("KEY")
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()
    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])
            

    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["swellHeight_m"]


def getWindDir(latlong):
    key = os.getenv("KEY")
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])

    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["winddir16Point"]


def getSwellDir(latlong):
    key = os.getenv("KEY")
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])

    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["swellDir16Point"]


def getWindSpeed(latlong):
    key = os.getenv("KEY")
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])

    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["windspeedKmph"]


def getSwellPeriod(latlong):
    key = os.getenv("KEY")
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])

    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["swellPeriod_secs"]


def getWaterTemp(latlong):
    key = os.getenv("KEY")
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()
    return data

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])

    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["waterTemp_C"]


def getWeatherDescription(latlong):
    key = os.getenv("KEY")
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()
    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)
    return current_time, data

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])

    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["weatherDesc"][0]["value"]

In [30]:
getWeatherDescription("9.8667,126.0500")

(2300,
 {'data': {'request': [{'type': 'LatLon', 'query': 'Lat 9.87 and Lon 126.05'}],
   'weather': [{'date': '2020-07-31',
     'astronomy': [{'sunrise': '05:26 AM',
       'sunset': '05:59 PM',
       'moonrise': '03:04 PM',
       'moonset': '02:00 AM',
       'moon_phase': 'Full Moon',
       'moon_illumination': '76'}],
     'maxtempC': '30',
     'maxtempF': '86',
     'mintempC': '26',
     'mintempF': '79',
     'hourly': [{'time': '0',
       'tempC': '27',
       'tempF': '80',
       'windspeedMiles': '11',
       'windspeedKmph': '18',
       'winddirDegree': '153',
       'winddir16Point': 'SSE',
       'weatherCode': '356',
       'weatherIconUrl': [{'value': 'http://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0026_heavy_rain_showers_night.png'}],
       'weatherDesc': [{'value': 'Moderate or heavy rain shower'}],
       'precipMM': '4.9',
       'precipInches': '0.2',
       'humidity': '83',
       'visibility': '7',
       'visibilityMiles': '4',
     

In [30]:
user_input = input("Input your current location, if possible enter your exact address:")
experience = input("Input your experience: ")


latlong = getCoords(user_input)
print(latlong)
longitude = latlong[0]
latitude = latlong[1]

df = getMoreBeaches(latitude,longitude, experience)
df['Swell Height'] = df['latlong'].apply(getSwellHeight)
df['Swell direction'] = df['latlong'].apply(getSwellDir)
df['Swell period'] = df['latlong'].apply(getSwellPeriod)
df['Wind speed'] = df['latlong'].apply(getWindSpeed)
df['Wind direction'] = df['latlong'].apply(getWindDir)
df['Water temperature'] = df['latlong'].apply(getWaterTemp) 
df['Weather Description'] = df['latlong'].apply(getWeatherDescription)

display(df)


Input your current location, if possible enter your exact address:North Shore, Oahu, Hawaii
Input your experience: Experienced
[-79.65345, 37.08168]


KeyError: 'Wave quality'

In [9]:
df = getMoreBeaches("43.3856", "-4.29151") 

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
df


,Name,Address,Direction,Experience,Wave quality,Frequency,Bottom,Best wind,Best swell,Latitude,Longitude,latlong
0,Comillas,"Playa de Comillas, Paseo de Jesús Cancio, Semi...",Right,Experienced surfers,Normal,Sometimes break,Reef-rocky,"[SouthWest, South]","[North, NorthWest, West]",43.389955,-4.288316,"43.389955,-4.288316"
1,Oyambre,"Playa de Oyambre, Ría del Capitán-San Vicente ...",Right and left,All surfers,Normal,Regular,Beach-break,"[SouthWest, South, SouthEast]","[NorthWest, West]",43.394633,-4.336081,"43.394633,-4.336081"
2,Meron,"Avenida de Francisco Giner de los Ríos, Barrio...",Right and left,All surfers,Regional Classic,Regular,Beach-break,[SouthEast],"[NorthWest, West]",43.390953,-4.380970,"43.390953,-4.38097"
3,Los Locos,"Playa de los Locos, Calle de Sobremar, Suances...",Right and left,All surfers,Regional Classic,Regular,Beach-break,"[East, NorthEast]","[NorthWest, West]",43.438695,-4.047754,"43.438695,-4.047754"
4,Playa de La Concha,"Playa de la Concha, Calle Doctor Enrique Otí, ...",Right and left,Beginners wave,Choss,Regular,Beach-break,"[SouthWest, South]","[NorthWest, West]",43.436202,-4.041445,"43.436202,-4.041445"
...,...,...,...,...,...,...,...,...,...,...,...,...
157,Le Bayle Nord,"Le Truc Vert, Lège-Cap-Ferret, Arcachon, Giron...",Right and left,All surfers,Normal,Regular,Beach-break,"[SouthEast, East]","[West, SouthWest]",44.721762,-1.250762,"44.721762,-1.250762"
158,Fornos,"Fornos Beach, DP-6116, Cariño, Ortegal, Galici...",Right and left,All surfers,Normal,Sometimes break,Sand-bar,"[West, SouthWest, South]",[NorthEast],43.713193,-7.853186,"43.713193,-7.853186"
159,Grand Crohot,"Plage du Grand Crohot, Accès Plage du Grand Cr...",Right and left,All surfers,Regional Classic,Regular,Beach-break,"[SouthEast, East, NorthEast]","[NorthWest, West, SouthWest]",44.797551,-1.234245,"44.797551,-1.234245"
160,La Jenny,"Plage de La Jenny, Piste Cyclable Lacanau-Lege...",Right and left,All surfers,Normal,Very consistent (150 day/year),Beach-break,"[SouthEast, East, NorthEast]","[NorthWest, West, SouthWest]",44.857268,-1.224031,"44.857268,-1.224031"


In [11]:
df["Latitude"][0]

43.389955